In [8]:
import warnings
warnings.filterwarnings('ignore')

from crewai import Agent, Task, Crew
from crewai_tools import SerperDevTool
from langchain_ollama import ChatOllama

import os
from utils import openaikey, serperkey

openai_api_key = openaikey()
serper_api_key = serperkey()

#For OpenAI Models:
os.environ["OPENAI_MODEL_NAME"] = 'ENTER MODEL NAME HERE'

#For locally hosted models with Ollama. To run with Ollama, specify to use this llm in the agent system.
llm = ChatOllama(
    model="MODEL NAME",
    base_url="http://localhost:11434"
)

In [5]:
def kickoffCrew(inputs):
    ###AGENTS###
    support_agent = Agent(
        role="Senior Support Representative",
        goal="Be the most friendly and helpful "
            "support representative in your team",
        backstory=(
            "You work at SearchR and "
            " are now working on providing "
            "support to {customer}, a super important customer "
            " for your company."
            "You need to make sure that you provide the best support!"
            "Make sure to provide full complete answers, "
            " and take care of the customer's needs."
        ),
        allow_delegation=False,
        verbose=True
    )
    
    ###TOOLS###
    web_search_tool = SerperDevTool()
    
    ###TASKS###
    inquiry_resolution = Task(
        description=(
            "{customer} just reached out with a super important ask:\n"
            "{inquiry}\n\n"
            "Make sure to use everything you know "
            "to provide the best support possible."
            "You must strive to provide a complete "
            "and accurate response to the customer's inquiry."
        ),
        expected_output=(
            "A detailed, informative response to the "
            "customer's inquiry that addresses "
            "all aspects of their question.\n"
            "The response should include references "
            "to everything you used to find the answer, "
            "including external data or solutions. "
            "Ensure the answer is complete, "
            "leaving no questions unanswered, and maintain a helpful and friendly "
            "tone throughout."
        ),
        tools=[web_search_tool],
        agent=support_agent, 
    )

    crew = Crew(
        agents=[support_agent],
        tasks=[inquiry_resolution],
        verbose=2,
        memory=True
    )

    res = crew.kickoff(inputs=inputs)
    return res.raw

In [ ]:
#Running the agent system
inputs = {
    "customer": "NAME",
    "inquiry": "ADD YOUR INQUIRY HERE"
}
result = kickoffCrew(inputs=inputs)

In [ ]:
#Display system output
from IPython.display import Markdown
Markdown(result)